<a href="https://colab.research.google.com/github/Khyatidaksha15/Filtering/blob/main/collaborativefiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('/content/IMDB_Top250Engmovies2_OMDB_Detailed.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Index              250 non-null    int64  
 1   Title              250 non-null    object 
 2   Year               250 non-null    int64  
 3   Rated              250 non-null    object 
 4   Released           248 non-null    object 
 5   Runtime            250 non-null    object 
 6   Genre              250 non-null    object 
 7   Director           250 non-null    object 
 8   Writer             249 non-null    object 
 9   Actors             250 non-null    object 
 10  Plot               250 non-null    object 
 11  Language           250 non-null    object 
 12  Country            250 non-null    object 
 13  Awards             245 non-null    object 
 14  Poster             250 non-null    object 
 15  Ratings.Source     250 non-null    object 
 16  Ratings.Value      250 non

In [4]:
df = df.dropna(axis=1, how='all')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Index           250 non-null    int64  
 1   Title           250 non-null    object 
 2   Year            250 non-null    int64  
 3   Rated           250 non-null    object 
 4   Released        248 non-null    object 
 5   Runtime         250 non-null    object 
 6   Genre           250 non-null    object 
 7   Director        250 non-null    object 
 8   Writer          249 non-null    object 
 9   Actors          250 non-null    object 
 10  Plot            250 non-null    object 
 11  Language        250 non-null    object 
 12  Country         250 non-null    object 
 13  Awards          245 non-null    object 
 14  Poster          250 non-null    object 
 15  Ratings.Source  250 non-null    object 
 16  Ratings.Value   250 non-null    float64
 17  Metascore       177 non-null    flo

In [7]:
import pandas as pd
import numpy as np

# Load IMDb data from a CSV file
# Replace 'imdb_data.csv' with the path to your actual CSV file
imdb_data = pd.read_csv('/content/IMDB_Top250Engmovies2_OMDB_Detailed.csv')

# Preview the dataset
print(imdb_data.head())

# Number of synthetic users to create
num_users = 1000

# Generate synthetic user IDs
user_ids = [f'user_{i}' for i in range(1, num_users + 1)]

# Define the number of ratings per user (e.g., each user rates 10-20 random movies)
ratings_per_user = np.random.randint(10, 20, size=num_users)

# Generate synthetic user-movie-rating data
synthetic_data = []

for user_id, num_ratings in zip(user_ids, ratings_per_user):
    # Randomly sample movies for this user
    sampled_movies = imdb_data.sample(n=num_ratings, replace=False)

    for _, movie in sampled_movies.iterrows():
        # Assign a random rating between 1 and 10
        rating = np.random.uniform(1, 10)
        synthetic_data.append([user_id, movie['Title'], rating])

# Create a DataFrame with synthetic user data
user_ratings_df = pd.DataFrame(synthetic_data, columns=['user_id', 'title', 'rating'])

# Display the synthetic dataset
print(user_ratings_df.head())


   Index                     Title  Year     Rated   Released  Runtime  \
0      1  The Shawshank Redemption  1994         R  14-Oct-94  142 min   
1      2             The Godfather  1972         R  24-Mar-72  175 min   
2      3    The Godfather: Part II  1974         R  20-Dec-74  202 min   
3      4           The Dark Knight  2008     PG-13  18-Jul-08  152 min   
4      5              12 Angry Men  1957  APPROVED  01-Apr-57   96 min   

                  Genre              Director  \
0          Crime, Drama        Frank Darabont   
1          Crime, Drama  Francis Ford Coppola   
2          Crime, Drama  Francis Ford Coppola   
3  Action, Crime, Drama     Christopher Nolan   
4          Crime, Drama          Sidney Lumet   

                                              Writer  \
0  Stephen King (short story "Rita Hayworth and S...   
1  Mario Puzo (screenplay), Francis Ford Coppola ...   
2  Francis Ford Coppola (screenplay), Mario Puzo ...   
3  Jonathan Nolan (screenplay), Chri

In [13]:
!pip install scikit-surprise

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [14]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(user_ratings_df[['user_id', 'title', 'rating']], reader)

In [15]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [16]:
svd = SVD()
svd.fit(trainset)

In [17]:
predictions = svd.test(testset)
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')

RMSE: 2.6634
RMSE: 2.66337156867304


In [18]:
user_id = 'user_17'  # Replace with any synthetic user ID from user_ratings_df
all_titles = user_ratings_df['title'].unique()


In [19]:
user_rated_titles = user_ratings_df[user_ratings_df['user_id'] == user_id]['title']
unrated_titles = [title for title in all_titles if title not in user_rated_titles.values]


In [20]:
recommendations = []
for title in unrated_titles:
    est_rating = svd.predict(user_id, title).est
    recommendations.append((title, est_rating))

recommendations.sort(key=lambda x: x[1], reverse=True)

In [21]:
print("Top 5 movie recommendations for user:", user_id)
for title, rating in recommendations[:5]:
    print(f"{title}: estimated rating {rating:.2f}")

Top 5 movie recommendations for user: user_17
The Bourne Ultimatum: estimated rating 7.57
Saving Private Ryan: estimated rating 7.39
Warrior: estimated rating 7.26
Hotel Rwanda: estimated rating 6.90
Kind Hearts and Coronets: estimated rating 6.87


In [11]:
def recommend_movies_for_user(user_id, num_recommendations=5):
    # Ensure the user exists in the dataset
    if user_id not in user_ratings_df['user_id'].unique():
        print(f"User ID '{user_id}' not found in dataset.")
        return

    # List of all movie titles in the dataset
    all_titles = user_ratings_df['title'].unique()

    # Find movies the user hasn't rated yet
    user_rated_titles = user_ratings_df[user_ratings_df['user_id'] == user_id]['title']
    unrated_titles = [title for title in all_titles if title not in user_rated_titles.values]

    # Predict ratings for unrated movies and gather recommendations
    recommendations = []
    for title in unrated_titles:
        est_rating = svd.predict(user_id, title).est
        recommendations.append((title, est_rating))

    # Sort recommendations by estimated rating in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Display top N recommendations
    print(f"Top {num_recommendations} movie recommendations for user '{user_id}':")
    for title, rating in recommendations[:num_recommendations]:
        print(f"{title}: estimated rating {rating:.2f}")

# Example usage
user_id_to_recommend = 'user_100'  # Replace with any user ID of your choice
recommend_movies_for_user(user_id_to_recommend, num_recommendations=8)


Top 8 movie recommendations for user 'user_100':
The Best Years of Our Lives: estimated rating 7.55
The Exorcist: estimated rating 7.49
Sleuth: estimated rating 7.25
Kill Bill: Vol. 1: estimated rating 7.04
The Bourne Ultimatum: estimated rating 7.03
Prisoners: estimated rating 7.02
Once Upon a Time in America: estimated rating 6.98
Rear Window: estimated rating 6.89


In [12]:
user_ratings_df.to_csv('synthetic_user_ratings.csv', index=False)

In [26]:
import pandas as pd
import numpy as np

# Load the synthetic user ratings dataset
user_ratings_df = pd.read_csv('synthetic_user_ratings.csv')

# Function to recommend movies based on genre
def recommend_movies_by_genre(user_id, genre, num_recommendations=5):
    # Get movies from the specified genre
    genre_movies = imdb_data[imdb_data['Genre'].str.contains(genre, case=False, na=False)]

    # Get movies rated by the user
    rated_movies = user_ratings_df[user_ratings_df['user_id'] == user_id]

    # Filter out movies that the user has already rated
    unrated_movies = genre_movies[~genre_movies['Title'].isin(rated_movies['title'])]

    # Sort by rating (assuming higher ratings are better)
    recommended_movies = unrated_movies.sort_values(by='imdbRating', ascending=False).head(num_recommendations)

    return recommended_movies[['Title', 'imdbRating', 'Genre']]

# Example usage
user_id = 'user_1'  # Specify the user ID for whom you want recommendations
genre = 'Action'     # Specify the genre for recommendations
recommendations = recommend_movies_by_genre(user_id, genre)

# Display recommended movies
print("Recommended Movies:")
print(recommendations)


Recommended Movies:
                                             Title  imdbRating  \
3                                  The Dark Knight         9.0   
12                                       Inception         8.8   
11  Star Wars: Episode V - The Empire Strikes Back         8.8   
16                                      The Matrix         8.7   
31                         Raiders of the Lost Ark         8.5   

                         Genre  
3         Action, Crime, Drama  
12   Action, Adventure, Sci-Fi  
11  Action, Adventure, Fantasy  
16              Action, Sci-Fi  
31           Action, Adventure  


In [27]:
user_id = 'user_100'  # Specify the user ID for whom you want recommendations
genre = 'Action'     # Specify the genre for recommendations
recommendations = recommend_movies_by_genre(user_id, genre)

# Display recommended movies
print("Recommended Movies:")
print(recommendations)


Recommended Movies:
                                             Title  imdbRating  \
3                                  The Dark Knight         9.0   
12                                       Inception         8.8   
11  Star Wars: Episode V - The Empire Strikes Back         8.8   
16                                      The Matrix         8.7   
17              Star Wars: Episode IV - A New Hope         8.7   

                         Genre  
3         Action, Crime, Drama  
12   Action, Adventure, Sci-Fi  
11  Action, Adventure, Fantasy  
16              Action, Sci-Fi  
17  Action, Adventure, Fantasy  
